<a href="https://colab.research.google.com/github/jordynjey/Aprendiendo_claculadora/blob/master/censpag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas.core.reshape.encoding import Index
from numpy.core.numeric import indices
#Librerias de lectura y rutas
from os import path
from pathlib import Path
from zmq import ROUTER_RAW
from typing import Container
#librerias http
import requests
import re
#libreria de scraping parsear codigo Html
from bs4 import BeautifulSoup
import lxml.html as html
#--------librerias de lectura y datos----
import numpy as np
import pandas as pd
from pandas import ExcelWriter
#Librerias para pasar al Excel#
from openpyxl import load_workbook,Workbook
from openpyxl.styles import Font
from datetime import datetime

#Pagina de CENS
url= 'https://www.cens.com.co'

#------------------------Primero-----------------------------#
pagi_cens=requests.get(url)
#conocer el estado de la conexión a la pagina
#pagi_cens.status_code
#imprimir estructura de la pagina
#print(pagi_cens.text)
#pagi_cens.content
#-----------------Funciones-----------------------#
def agregar_links_etiquetas_a(atributo,lista):
  lista=[s.get('href') for s in atributo]

def impresion_lista(lista_im):
  for i in lista_im:
    print(i)

def agregar_datos(lista_recorre,lista_agrega):
  for i in lista_recorre:
    lista_agrega.append(i)
  return lista_agrega

def quitar_duplicados_lista(lista_com):
  sin_duplica_links=[]
  for dato in lista_com:
    if dato not in sin_duplica_links:
      sin_duplica_links.append(dato)
  print('Se eliminaron : '+  str(len(lista_com)-len(sin_duplica_links))+ ' duplicados')
  #lista_com=sin_duplica_links
  return sin_duplica_links

def arreglar_relativas(lista):
  cambios=0
  indice=0
  for i in lista:
    li=str(i)
    if str(li).startswith('/'):
      #print('Antes de cambiar  ' , li)
      indice=lista.index(i)
      cadena=url+li
      #print('Despues de cambiar', cadena)
      lista[indice]=cadena
      cambios+=1
  print('Cambios realizados: '+ str(cambios))
  return lista

def quitar_restriciones(lista,listarestr):
    lista2=[]
    for a in lista:
      if a not in listarestr:
        lista2.append(a)
    lista=lista2
    return lista


def quitar_restriciones_segundo_nvel(lista,listarestr):
  lista2=[]
  for a in lista:
    bolean=False
    bolean=a.startswith('javascript')
    bolean2=a.startswith('/Portals/')
    bolean3=a.startswith('#tpt')
    if bolean==False:
      if bolean2==False:
        if bolean3==False:
          if a not in listarestr:
            lista2.append(a)
  lista=lista2
  return lista

#creación de sopa de pagina principal
soup=BeautifulSoup(pagi_cens.text,'lxml')
# Imprimir de pretiffy es
#print(soup_main.prettify())

#find traer el primer atributo con las condiciones que desee colocar
#find_all trae todos las etiquetas que solicite
atributo=soup.findAll('a')
atributo2=soup.find_all('div')
linksprueba=[se.get('href') for se in atributo]

#print(list(enumerate(atributo)))
#atributo=soup_main.find_all('a', attrs={"target":'_blank'})
#atributo=soup_main.find_all('a', attrs={"target":""})
#print(len(atributo))
#atributo= soup_main.find_all('href')
#Para saber numero de X etiquetas
len(soup.find_all(href=True))

#------------------------Segundo-----------------------------#
#--limpiar la información para poderla recorrer sin errores--#
restricciones=["javascript:__doPostBack('dnn$dnnSearch$cmdSearch','')"]
#--Se lee archivo de restricciones y se unen con las que ya se tienen de manera tecnica--#
path="/content/sample_data/links_restringido.xlsx"
openfile=pd.read_excel(path,engine='openpyxl',sheet_name='Sheet')
#Restricciones tecnicas o que no se van tomar en cuenta
links_restringidos=['#','',None,'https://gobiernodigital.mintic.gov.co/portal/','https://www.tiktok.com/@censgrupoepm',
                    'https://www.linkedin.com/company/64512425/admin/','https://www.youtube.com/user/buenasenergiascens' ,
                    'https://twitter.com/CENSGrupoEPM?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor' ,
                    'https://www.instagram.com/censgrupoepm/?hl=es-la','https://apps.apple.com/co/app/cens-app-movil/id1357634972',
                    'https://play.google.com/store/apps/details?id=com.cens.app.movil&hl=es_CO&gl=US','018000522955','018000414115',
                    '(607)5824444','115','javascript:void(0);','javascript:void(0)','https://www.gov.co/home/','https://www.colombia.co/',
                    'https://gobiernodigital.mintic.gov.co/portal','/institucional/transparencia/normatividad','/terminos-y-condiciones',
                    '/institucional/proteccion-de-datos-personales',"javascript:__doPostBack('dnn$ctr703$Login$Login_DNN$cmdLogin','')",
                    '#content','comercial@cooguasimales.com','eventos@cooguasimales.com','talentohumano@cooguasimales.com','mailto:soporte.clientes@cens.com.co',
                    'comercial@cooguasimales.com','eventos@cooguasimales.com','talentohumano@cooguasimales.com','https://www.veolia.com/latamib/es',
                    'https://biogersaesp.com/','https://www.coomultrasan.com.co/','https://www.ecoambientaldelnorte.com/','https://electriya.com/',
                    'https://www.almacenesrayco.com/','https://web.facebook.com/Electrogases-SAS-2333394940231027/','https://emzulia.com/','https://geoaseo.co/',
                    'https://www.cucuta.losolivos.co/']


datosarchivorestric=openfile.values
links_restringidos=agregar_datos(datosarchivorestric,links_restringidos)
print('Datos restringidos leidos')
links_restringidos.append("javascript:__doPostBack('dnn$dnnSearch$cmdSearch','')")
links=[se.get('href') for se in atributo]
links=quitar_restriciones(links,links_restringidos)
print('Restricciones eliminados de los links')
#Se lee archivo de restricciones y anexa#
print('Tamaño con duplicados:  '+ str(len(links)))
links=quitar_duplicados_lista(links)
print('Tamaño sin duplicados:  '+ str(len(links)))
#[atribu.get('href') for atribu in links if atribu.get('href').startswith('/') ==True ]
print('Arreglando rutas relativas')
links=arreglar_relativas(links)
print('Rutas relativas arregladas')
#impresion_lista(links)
#Pasar los links a una hoja de excel#
libro=Workbook()
sheet=libro.active
#sheet['B1']='Item'
sheet['A1']= 'Link'
fila=2 #Fila inicia
col_uno=1 #Columnnas de links
col_dato=2

for dato in links:
  #sheet.cell(column=col_uno,row=fila, value=fila-1)
  sheet.cell(column=col_uno,row=fila,value=dato)
  fila=fila+1

libro.save('prueba1.xlsx')
path1="/content/prueba1.xlsx"
openfile1=pd.read_excel(path1,engine='openpyxl',sheet_name='Sheet')
datosarchivolec=openfile1.values
print(links[115])

#-----Links de segundo nivel-----#
print('----------------------Comienza segundo nivel----------------------')
listaacumula=[]

pagiinter=''
for i in links:
  j=links.index(i)
  print(links[j])
  if j<115:
    links2=links[j]
    pagiinter=requests.get(links[j])
    sopa=BeautifulSoup(pagiinter.text,'lxml')
    atributo3=sopa.findAll('a')
    listax=[se.get('href') for se in atributo3]
    #listax=quitar_restriciones(listax,links_restringidos)
    #print('Imprimiendo lista X')
    #print(listax)
    listaacumula=agregar_datos(listax,listaacumula)
    print('Tamaño de la lista acumuladora: ' + str(len(listaacumula)))
    #print(links2 + '  codigo:'  + str(pagiinter.status_code)+ '  posición ' + str(j))
    int(j)

print('Termino el recorrido')
listaacumula=quitar_restriciones(listaacumula,links_restringidos)
print('Quitar restricciones 2')
print("Sin quitar los java: "+str(len(listaacumula)))
listaacumula=quitar_restriciones_segundo_nvel(listaacumula,links_restringidos)
print("Quitados los java: " + str(len(listaacumula)))
listaacumula=quitar_duplicados_lista(listaacumula)
print(len(listaacumula))
impresion_lista(listaacumula)



def links_in_links(links):
  linksfor=[]
  for i in links:
    if links.index(i)<115:
      #print(i)
      pagiin=requests.get(i)
      soup_inter=BeautifulSoup(pagiin.text,'lxml')
      atri_inter=soup_inter.find_all('a')
      linksin=[linkint.get('href') for linkint in atri_inter if linkint.get('href') != '' and linkint.get('href')!="#"]
      #Imprimiendo el tamaño
      links_in_princ.append(linksin)
      #print(i)
      #print(len(linksin))







Datos restringidos leidos
Restricciones eliminados de los links
Tamaño con duplicados:  162
Se eliminaron : 24 duplicados
Tamaño sin duplicados:  138
Arreglando rutas relativas
Cambios realizados: 5
Rutas relativas arregladas
https://censvirtual.sistemasentry.com.co/VisionWebApp/
----------------------Comienza segundo nivel----------------------
https://www.cens.com.co/transparencia-y-acceso-a-la-informacion-publica-2021
Tamaño de la lista acumuladora: 330
https://www.cens.com.co/servicios-in-linea
Tamaño de la lista acumuladora: 340
https://www.cens.com.co/clientes-y-usuarios/tramites-y-servicios
Tamaño de la lista acumuladora: 618
https://www.cens.com.co/radica-tu-pqr
Tamaño de la lista acumuladora: 833
https://www.cens.com.co/participacion-ciudadana
Tamaño de la lista acumuladora: 1107
https://www.cens.com.co/
Tamaño de la lista acumuladora: 1348
https://www.cens.com.co/home/informes-de-sostenibilidad
Tamaño de la lista acumuladora: 1607
https://www.cens.com.co/home/eventos-y-patroc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#------------------EXCEL-----------------#
#Librerias para pasar al Excel#
from openpyxl import load_workbook,Workbook
from openpyxl.styles import Font
from datetime import datetime

#Pasar los links a una hoja de excel#
libro=Workbook()
sheet=libro.active
sheet['A1']='Jordyn'
libro.save('prueba1.xlsx')
print('Se creo el archivo')
#--Leyendo nuevamente el archivo---#
path="/content/links_restringido.xlsx"
openfile=pd.read_excel(path,engine="openpyxl",sheet_name="Sheet")
datosarchivorestric=openfile.values
for i in datosarchivorestric:
  print(i)

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 150  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


In [ ]:
from IPython.utils.text import string
links_restringidos=['#','',None,'https://gobiernodigital.mintic.gov.co/portal/','https://www.tiktok.com/@censgrupoepm',
                    'https://www.linkedin.com/company/64512425/admin/','https://www.youtube.com/user/buenasenergiascens' ,
                    'https://twitter.com/CENSGrupoEPM?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor' ,
                    'https://www.instagram.com/censgrupoepm/?hl=es-la','https://apps.apple.com/co/app/cens-app-movil/id1357634972',
                    'https://play.google.com/store/apps/details?id=com.cens.app.movil&hl=es_CO&gl=US','018000522955','018000414115',
                    '(607)5824444','115','javascript:void(0);','javascript:void(0)','https://www.gov.co/home/','https://www.colombia.co/',
                    'https://gobiernodigital.mintic.gov.co/portal','/institucional/transparencia/normatividad','/terminos-y-condiciones',
                    '/institucional/proteccion-de-datos-personales']

for i in links_restringidos:
  if str(i).startswith('#'):
    print(i)

#
